In [2]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from nltk.tokenize import RegexpTokenizer

from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
import nltk
from collections import Counter
import pandas as pd
import numpy as np

In [3]:
new_path = '/home/cloudera/cs2.txt'
def read_file(str):
    file = open(str,'r')
    txt=file.read()
    #print(txt)
    return txt
txt=read_file(new_path)

In [4]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs
pragraphs=txt_pragraphs(txt)
type(pragraphs)
print ("Number of Paragraphs",len(pragraphs))

Number of Paragraphs 7


In [5]:
doc_set=[]
for index in range(len(pragraphs)):
    doc_set.append(pragraphs[index])

In [7]:
#http://programtalk.com/python-examples/sklearn.decomposition.NMF/



#https://de.dariah.eu/tatom/topic_model_python.html

pragraphs_names=[]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
    
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
english_stops = set(stopwords.words('english'))

#NUM_TOPICS = 5
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    

# list for tokenized documents in loop
texts = []
#print(len(doc_set))
words_nums=0
# loop through document list
x=0
all_words=[]
idx_pragraph=0
for i in doc_set:#pragraphs list
    
    pragraphs_names.append('Pragraph_no_'+str(idx_pragraph))
    
    idx_pragraph += 1
    # clean and tokenize document string
    raw = i.lower()
    #tokens = tokenizer.tokenize(raw)
    sentences_pragraph=sent_tokenize(raw)
    
    # remove stop words from tokens
    #stopped_tokens = [i for i in tokens if not i in en_stop]
    words_pragraph=''
    for s in sentences_pragraph:
        tokens = tokenizer.tokenize(s)
        stopped_tokens = [s for s in tokens if not s in en_stop and not s in new_stop_words and not s in new_stop_words2 and not s in english_stops]
        #print("stopped_tokens \n",stopped_tokens)
        words_nums+=len(stopped_tokens)
        #print(stopped_tokens)
        #lesk_sentence=[]
        wordsent=''
        for word in stopped_tokens:
            lesk_word=lesk_synset=lesk(s,word, 'n')
            
            if lesk_word is not None:
                #print(lesk_word)
                wordsent+=' '+str(lesk_word)
                words_pragraph+=' '+"'"+lesk_word.name()+"'"
                all_words.append(lesk_word.name())
              
    x=x+1
    
   
    texts.append(words_pragraph)
    
    


print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w+\\b\\.\\w\\.\\d\\d')
#print(tfidf_vectorizer)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(texts)
#print(tfidf)
print("done in %0.3fs." % (time() - t0))

A = tfidf_vectorizer.fit_transform(texts)

print(A.shape)




lda = LatentDirichletAllocation()
topic_document = lda.fit_transform(A)# W
topic_word_matrix = []
document_topic_matrix = []
vocabulary_size = len(all_words)
#print("2",len(all_words))
row = []
col = []
data = []
data_rows=[]
topic_name=[]
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
#print('fff',len(tfidf_feature_names))
for topic_idx, topic in enumerate(lda.components_):#word topic H
    
    topic_row=[]
    topic_arg_sort=topic.argsort()
    for i in range(len(topic_arg_sort)):
        row.append(topic_idx)
        col.append(i)
        data.append(topic[i])
        topic_row.append(topic[i])
    data_rows.append(topic_row)
    topic_name.append('Topic_no_'+str(topic_idx))
       
row = []
col = []
data = []
doc_count = 0
data_rows_Doc=[]

for doc in topic_document: # topic Document W
    topic_count = 0
    
    topic_row=[]
    doc_arg_sort=doc.argsort()
    for topic_weight in doc_arg_sort:
        row.append(doc_count)
        col.append(topic_count)
        data.append(topic_weight)
        topic_row.append(topic_weight)
    
        
        topic_count += 1
    data_rows_Doc.append(topic_row)
   
    doc_count += 1






1 491
Extracting tf-idf features for NMF...
done in 0.016s.
(7, 196)


/usr/local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [8]:
df_word_topic=pd.DataFrame(data_rows,columns=tfidf_feature_names,index=topic_name)

df_word_topic

,ace.n.03,acquiring.n.01,algorithm.n.01,automaton.n.02,beginning.n.05,bind.n.01,broad.n.01,building.n.04,business.n.04,calculation.n.02,...,view.n.02,view.n.07,way.n.12,well.n.05,whitethorn.n.01,wish.n.01,workplace.n.01,world.n.02,world.n.08,young.n.04
Topic_no_0,0.279771,0.261438,0.268243,0.266782,0.284626,0.262361,0.259698,0.242841,0.279245,0.317041,...,0.246369,0.273972,0.343489,0.249193,0.282387,0.276594,0.271741,0.292751,0.274287,0.322127
Topic_no_1,0.263069,0.283264,0.242272,0.287946,0.281378,0.275091,0.261156,0.299691,0.295299,0.282382,...,0.316235,0.279290,0.311538,0.274457,0.281880,0.296245,0.301199,0.285569,0.273488,0.287331
Topic_no_2,0.289356,0.279622,0.325175,0.314120,0.240412,0.233301,0.253496,0.274401,0.291366,0.274019,...,0.297179,0.311650,0.289742,0.225533,0.279077,0.279737,0.273444,0.260227,0.246533,0.266357
Topic_no_3,0.271365,0.273450,0.247951,0.237710,0.288142,0.281682,0.275569,0.286034,0.251931,0.276816,...,0.284023,0.268187,0.271029,0.246156,0.280920,0.299968,0.269344,0.304757,0.296165,0.306062
Topic_no_4,0.251794,0.291410,0.276379,0.268338,0.255870,0.274312,0.305307,0.278598,0.290967,0.283627,...,0.278926,0.299983,0.286213,0.257923,0.284548,0.304759,0.288429,0.289954,0.268630,0.300627
Topic_no_5,0.271233,0.274945,0.274621,0.312966,0.287367,0.303753,0.246130,0.274375,0.260125,0.298181,...,0.278841,0.276641,0.245812,0.282925,0.267977,0.273714,0.271783,0.291046,0.272208,0.294661
Topic_no_6,0.309222,0.290602,0.290716,0.282799,0.289893,0.263696,0.283041,0.264532,0.265550,0.275010,...,0.268119,0.246763,0.258332,0.261602,0.264152,0.290038,0.257141,0.284691,0.275550,0.291668
Topic_no_7,0.270946,0.279471,0.253102,0.427337,0.271800,0.278628,0.253815,0.278512,0.296800,0.296176,...,0.272099,0.244323,0.281988,0.280971,0.420113,0.275856,0.291532,0.308622,0.520207,0.277962
Topic_no_8,0.296507,0.280289,0.415635,0.296550,0.359123,0.359916,0.297290,0.272497,0.279626,0.258311,...,0.373542,0.374944,0.380974,0.324265,0.273115,0.269427,0.234148,0.281273,0.338056,0.370109
Topic_no_9,0.347986,0.347990,0.398334,0.283391,0.239365,0.269412,0.342622,0.364686,0.338980,0.556378,...,0.275698,0.274811,0.272591,0.410171,0.327857,0.332553,0.423461,0.307509,0.282432,0.238505


In [9]:
df_doc_topic=pd.DataFrame(data_rows_Doc,columns=topic_name,index=pragraphs_names)

df_doc_topic

,Topic_no_0,Topic_no_1,Topic_no_2,Topic_no_3,Topic_no_4,Topic_no_5,Topic_no_6,Topic_no_7,Topic_no_8,Topic_no_9
Pragraph_no_0,3,6,5,4,1,0,7,2,8,9
Pragraph_no_1,3,0,1,5,6,4,7,2,9,8
Pragraph_no_2,3,0,1,6,5,4,2,7,8,9
Pragraph_no_3,6,3,0,5,1,4,7,8,9,2
Pragraph_no_4,0,6,3,1,5,2,7,8,9,4
Pragraph_no_5,3,6,0,5,1,2,4,8,9,7
Pragraph_no_6,6,5,3,1,0,4,2,9,7,8
